# Encoder-decoder scaled dot-product attention

Recordamos la arquitectura del `Scaled Dot-Product Attention`

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Ya lo explicamos, pero ahora lo volvemos a explicar teniendo en cuenta que `K` y `V` es la matriz que proviene del encoder y `Q` proviene del decoder, por lo que se realizará una atención entre el encoder y el decoder

# MatMul

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_decoder_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
</div>

Como ahora tenemos que `K` y `V` provienen del encoder, las llamaré $X_E$ y como `Q` proviene del decoder la llamaré $X_D$

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_first_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Veamos cómo serían sus matrices, en primer lugar `K` y `V` o $X_E$

$$K=V=X_E = \begin{pmatrix}
v_{E,1} \\
v_{E,2} \\
\vdots\\
v_{E,m} \\
\end{pmatrix} = \begin{pmatrix}
v_{E,1,1} & v_{E,1,2} & \cdots & v_{E,1,n} \\
v_{E,2,1} & v_{E,2,2} & \cdots & v_{E,2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{E,m,1} & v_{E,m,2} & \cdots & v_{E,m,n} \\
\end{pmatrix}$$

Y ahora `Q` o $X_D$

$$K=V=X_D = \begin{pmatrix}
v_{D,1} \\
v_{D,2} \\
\vdots\\
v_{D,m} \\
\end{pmatrix} = \begin{pmatrix}
v_{D,1,1} & v_{D,1,2} & \cdots & v_{D,1,n} \\
v_{D,2,1} & v_{D,2,2} & \cdots & v_{D,2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m,1} & v_{D,m,2} & \cdots & v_{D,m,n} \\
\end{pmatrix}$$

Por lo que la multiplicación entre `Q` y `K`, es decir, entre $X_E$ y $X_D$ es

$$X_D \cdot X_E^T = \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}$$

La multiplicación sera una multiplicación de matrices de dimensiones $\left(m_D \times n_D\right) \cdot \left(n_E \times m_E\right)$, por lo que para que se pueda producir $n_D = n_E$, donde $n_D$ es la dimensión del embedding del decoder y $n_E$ es la dimensión del embedding del encoder. Es decir, para poder realizar esta operación, tanto el embedding del encoder como del decoder tienen que tener la misma dimensión.

Si ambas dimensiones de embedding son iguales, obtendremos como resultado una matriz de tamaño $\left(m_D \times m_E\right)$ donde $m_D$ era el número de tokens de la frase del decoder y $m_E$ era el número de tokens de la frase del encoder

Representamos la dimensión

$$X_D \cdot X_E^T = \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}$$

## Scale

A continuación se divide entre la dimensión del embedding de `K`, es decir, del encoder, pero en realidad da igual, porque hemos visto que la dimnesión del embedding del encoder y del decoder tienen que ser iguales. Esto se hace por ser una normalización `norma L2`

Así que nos queda

$$
\text{Scale} = \frac{1}{\sqrt{d_k}} \cdot \left( X_D \cdot X_E^T \right) = \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}
$$

## Mask

En este módulo de `Multi-Head Attention` no se realiza enmascaramiento, ya que el enmascaramiento se realiza para enmascarar el futuro de la secuencia de salida. Y eso ya lo hemos hecho en el anterior módulo de atención

## Softmax

Realizamos el `Softmax` de la amtriz que tenemos

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_softmax.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Por lo que nos quedaría una matriz así

$$
\text{Softmax} = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \left( X_D \cdot X_E^T \right) \right) = \\
 = \text{softmax}\left( \frac{1}{\sqrt{d_k}} \cdot \begin{pmatrix}
v_{D,1} \cdot v_{E,1} & v_{D,1} \cdot v_{E,2} & \cdots & v_{D,1} \cdot v_{E,m} \\
v_{D,2} \cdot v_{E,1} & v_{D,2} \cdot v_{E,2} & \cdots & v_{D,2} \cdot v_{E,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{D,m} \cdot v_{E,1} & v_{D,m} \cdot v_{E,2} & \cdots & v_{D,m} \cdot v_{E,m} \\
\end{pmatrix}_{\left(m_D \times m_E\right)} \right)
$$

La cual podemos simplemente suponer como porcentajes de atención de los tokens salientes del encoder con los tokens del decoder

$$
\text{Softmax} = \begin{pmatrix}
p_{1D,1E} & p_{1D,2E} & \cdots & p_{1D,mE} \\
p_{2D,1E} & p_{2D,2E} & \cdots & p_{2D,mE} \\
\vdots & \vdots & \ddots & \vdots \\
p_{mD,1E} & p_{mD,2E} & \cdots & p_{mD,mE} \\
\end{pmatrix}_{\left(m_D \times m_E\right)}
$$

Por ejemplo en la traducción de `¿Cuál es ti nombre?` a `What is your name` el porcentaje de atención entre `nombre` y `name` debería ser muy alto, pues esta matriz representa la atención entre los tokens de la frase en español con los tokens de la frase en ingles

## MatMul

Por último volvemos a realizar un `MatMul` entre la matriz que tenemos que es de dimensiones $\left(m_D \times m_E\right)$ por `V` que como hemos dicho proviene de la salida del encoder ($X_E$), por lo que tiene dimensiones $\left(m_E \times n_E\right)$. Así que la multiplicación va a ser de dimensiones $\left(m_D \times m_E\right)·\left(m_E \times n_E\right)$, lo que nos da una matriz de $\left(m_D \times n_E\right)$, pero como las dimensiones del embedding del encoder y del decoder tienen que ser iguales nos queda una matriz de tamaño $\left(m_D \times m_D\right)$

Esto nos hace ver que hemos hecho lo correcto, porque si recuerdas, en el encoder siempre trabajamos con matrices $\left(m_E \times n_E\right)$, en el módulo de atención del encoder entraba una matriz de tamaño $\left(m_E \times n_E\right)$ y salía una matriz de tamaño $\left(m_E \times n_E\right)$. Incluso en el encoder entero entraba una matriz de tamaño $\left(m_E \times n_E\right)$ y salía una matriz de tamaño $\left(m_E \times n_E\right)$.

Por lo que ahora en si al decoder le entra una matriz $\left(m_D \times n_D\right)$ y a la salida de este segundo módulo de atención tenemos una matriz de tamaño $\left(m_D \times n_D\right)$ es que hemos hecho bien las cosas

Si realizamos la operación

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_second_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

En realidad lo que tenemos es la siguiente matriz

$$
\text{Matmul} = \begin{pmatrix}
p_{1D,1E} & p_{1D,2E} & \cdots & p_{1D,mE} \\
p_{2D,1E} & p_{2D,2E} & \cdots & p_{2D,mE} \\
\vdots & \vdots & \ddots & \vdots \\
p_{mD,1E} & p_{mD,2E} & \cdots & p_{mD,mE} \\
\end{pmatrix}_{\left(m_D \times m_E\right)} \cdot \begin{pmatrix}
v_{E,1} \\
v_{E,2} \\
\vdots\\
v_{E,m} \\
\end{pmatrix} = \\
 = \begin{pmatrix}
p_{1D,1E}·v_{E,1} + p_{1D,2E}·v_{E,2} + \cdots + p_{1D,mE}·v_{E,m} \\
p_{2D,1E}·v_{E,1} + p_{2D,2E}·v_{E,2} + \cdots + p_{2D,mE}·v_{E,m} \\
\vdots \\
p_{mD,1E}·v_{E,1} + p_{mD,2E}·v_{E,2} + \cdots + p_{mD,mE}·v_{E,m} \\
\end{pmatrix}
$$

Que representa

 * La primera fila (que representaría el primer token) se corresponde a la suma de probabilidades de atención del primer token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder
 * La segunda fila (que representaría al segundo token) se corresponde a la suma de probabilidades de atención del segundo token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder
 * Así sucesivamente, hasta la última fila (que representaría al último token) que se corresponde a la suma de probabilidades del último token del decoder con el resto de tokens del encoder por los embeddings del resto de tokens del encoder

# Implementación

La clase que hemos hecho hasta ahora para el `Scaled Dot-Product Attention` nos vale, ya que las operaciones son válidas, solo que tendremos que tener en cuenta que cuando la usemos, en este caso `K` y `V` tendrán que ser la matriz que sale del encoder y `Q` la matriz del decoder

In [ ]:
import torch
import torch.nn as nn
import math

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output